# Analyze EIBP Log Results

## Input Required Information

| Variable | Use |
| --- | --- |
| LOG_DIR_PATH | Location of the log directory. |

In [ ]:
LOG_DIR_PATH = "/home/fabric/work/FABRIC-Automation-main/logs"

In [ ]:
import re
import os

# initialize variables for calculating sum of eth_size, checking churn, and finding max conv_time
max_convergence_time = float('-inf')
sum_eth_size = 0
churn_count = 0
total_files = 0
min_down_time = float('+inf')

# define a regular expression to match the required lines
pattern = re.compile(r'^CONV_TIME:(\d+\.\d+)$')

# get the value for down_time from the log files
down_time_pattern = re.compile(r'^IF_TIMER_DOWN:(\d+\.\d+)$')
dir_down_time_pattern = re.compile(r'^IF_DIRECT_DOWN:(\d+\.\d+)$')


# loop through all files in the current directory
for filename in os.listdir(LOG_DIR_PATH):
    if filename.endswith('.log'):
        # open the file and loop through each line
        with open(os.path.join(LOG_DIR_PATH,filename), 'r') as file:
            for line in file:
                # check if the line matches the down_time pattern
                match = down_time_pattern.match(line.strip())
                match1 = dir_down_time_pattern.match(line.strip())
                if match:
                    # extract the down_time value from the line
                    down_time = float(match.group(1))
                    # update min_down_time if necessary
                    if down_time < min_down_time:
                        min_down_time = down_time

                if match1:
                    # extract the down_time value from the line
                    down_time = float(match1.group(1))
                    # update min_down_time if necessary
                    if down_time < min_down_time:
                        min_down_time = down_time

down_time = min_down_time

# loop through all files in the current directory again to calculate the other values
for filename in os.listdir(LOG_DIR_PATH):
    if filename.endswith('.log'):
        total_files += 1
        # open the file and loop through each line
        with open(os.path.join(LOG_DIR_PATH,filename), 'r') as file:
            # initialize variables for this file
            file_sum_eth_size = 0
            file_churn = False
            file_max_conv_time = float('-inf')
            for line in file:
                # check if the line matches the pattern
                match = pattern.match(line.strip())
                if match:
                    # get the next line that contains ETH_SIZE
                    eth_size_line = file.readline().strip()
                    # extract the values from the lines
                    conv_time = float(match.group(1))
                    eth_size = int(eth_size_line.split(':')[1])
                    # add eth_size to file_sum if conv_time is greater than down_time
                    if conv_time > down_time:
                        file_sum_eth_size += eth_size
                        # check if the next line contains CHURN_TRUE
                        churn_true_line = file.readline().strip()
                        if churn_true_line == 'CHURN_TRUE':
                            file_churn = True
                    # update file_max_conv_time if necessary
                    if conv_time > file_max_conv_time:
                        file_max_conv_time = conv_time
            # update global variables with values for this file
            if file_max_conv_time > max_convergence_time:
                max_convergence_time = file_max_conv_time
            sum_eth_size += file_sum_eth_size
            if file_churn:
                churn_count += 1

# calculate percentage of files with churn true
if total_files > 0:
    churn_percent = (churn_count / total_files) * 100
else:
    churn_percent = 0

# calculate the difference between max_conv_time and down_time
convergence_time = max_convergence_time - down_time

# display the result
print(f"CONVERGENCE_TIME: {convergence_time} s")
print(f"OVERHEAD: {sum_eth_size} bytes")
print(f"CHURN PERCENTAGE: {churn_percent:.2f}%")

CONVERGENCE_TIME: 0.5672049522399902 s
OVERHEAD: 1472 bytes
CHURN PERCENTAGE: 80.00%
